In [1]:
#!pip install cadCAD==0.4.23 &> /dev/null
#!pip uninstall numpy
#!pip uninstall pandas

#!pip install --user numpy
#!pip install --user  pandas

# Dependencies

In [2]:
# cadCAD standard dependencies

# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

# cadCAD global simulation configuration list
from cadCAD import configs

In [3]:
# Additional dependencies

# For parsing the data from the API
import json
# For downloading data from API
import requests as req
# For generating random numbers
import math
# For analytics
import pandas as pd
# For visualization
import plotly.express as px

## Query the Balancer subgraph for a UNI-BAL pool

In [4]:
# You can explore the subgraph at https://thegraph.com/explorer/subgraph/balancer-labs/balancer
# Pool address can be found on Etherscan
API_URI = 'https://api.thegraph.com/subgraphs/name/balancer-labs/balancer'

# Query for retrieving the history of swaps on a BAL <> UNI 50-50 pool
GRAPH_QUERY = '''
{
  swaps (where: {poolAddress: "0xf3168b50751173e150af543997abe0fec5d58b7f"}) { 
    tokenAmountIn,
    tokenInSym,
    tokenAmountOut,
    tokenOutSym,
    timestamp
  }
}
'''

# Retrieve data from query
JSON = {'query': GRAPH_QUERY}
r = req.post(API_URI, json=JSON)
graph_data = json.loads(r.content)['data']

print("Print first 500 characters of the response")
print(r.text[:500])

Print first 500 characters of the response
{"data":{"swaps":[{"timestamp":1606030017,"tokenAmountIn":"348","tokenAmountOut":"78.737271727059410272","tokenInSym":"UNI","tokenOutSym":"BAL"},{"timestamp":1610476730,"tokenAmountIn":"87.907993216103989248","tokenAmountOut":"254.406534991955029855","tokenInSym":"BAL","tokenOutSym":"UNI"},{"timestamp":1611736783,"tokenAmountIn":"40.890844784118336934","tokenAmountOut":"61.294208877262298814","tokenInSym":"BAL","tokenOutSym":"UNI"},{"timestamp":1618859828,"tokenAmountIn":"112.259419481903871425"


## Data Wrangle the Data

In [5]:
raw_df = pd.DataFrame(graph_data['swaps'])

raw_df.head(5)

,timestamp,tokenAmountIn,tokenAmountOut,tokenInSym,tokenOutSym
0,1606030017,348,78.737271727059410272,UNI,BAL
1,1610476730,87.907993216103989248,254.406534991955029855,BAL,UNI
2,1611736783,40.890844784118336934,61.294208877262298814,BAL,UNI
3,1618859828,112.259419481903871425,195.629827478447515933,BAL,UNI
4,1601954527,138.245503251295453184,27.894370921499086237,UNI,BAL


In [6]:
# Clean the data:
# 1. convert the raw timestamps to Python DateTime objects
# 2. make the token flow values numerical
# 3. order by time
df = (raw_df.assign(timestamp=lambda df: pd.to_datetime(df.timestamp, unit='s'))
            .assign(tokenAmountIn=lambda df: pd.to_numeric(df.tokenAmountIn))
            .assign(tokenAmountOut=lambda df: pd.to_numeric(df.tokenAmountOut))
            .sort_values('timestamp')
            .reset_index()
      )

df.head(5)

,index,timestamp,tokenAmountIn,tokenAmountOut,tokenInSym,tokenOutSym
0,76,2020-09-19 15:11:26,213.163101,78.928218,UNI,BAL
1,73,2020-09-21 18:58:57,53.681512,185.873296,BAL,UNI
2,44,2020-09-24 00:26:18,52.072207,156.679612,BAL,UNI
3,57,2020-09-24 12:54:53,203.769109,69.935233,UNI,BAL
4,64,2020-09-24 22:49:34,17.135790,50.760343,BAL,UNI


# Modelling

## 1. State Variables

In [7]:
initial_state = {
    # Cumulative transaction fees paid for each token
    'cumulative_fee_BAL': 0.0,
    'cumulative_fee_UNI': 0.0,
    
    # Cumulative swap volume
    # (positive means more tokens locked)
    # (negative means less tokens locked)
    'cumulative_swap_BAL': 0.0,
    'cumulative_swap_UNI': 0.0   
}
initial_state

{'cumulative_fee_BAL': 0.0,
 'cumulative_fee_UNI': 0.0,
 'cumulative_swap_BAL': 0.0,
 'cumulative_swap_UNI': 0.0}

## 2. System Parameters

In [8]:
# Transform the swap history data frame into a {timestep: data} dictionary
swap_dict = df.to_dict(orient='index')

system_params = {
    'swap_sequence': [swap_dict],
    
    # Transaction fees being applied to the input token
    'transaction_fee': [0.01, 0.05]
}

# Element for timestep = 3

## 3. Policy Functions

In [9]:
def p_swap(params, substep, state_history, previous_state):
    """
    Calculate cumulative transaction fees & swaps
    from a swap event
    """
    t = previous_state['timestep']
    
    # Data for this timestep
    ts_data = params['swap_sequence'][t]    
    
    timestamp = ts_data['timestamp']
    # Swap amounts
    # swap_in: Swap being locked to the pool
    swap_in = ts_data['tokenAmountIn']
    # swap_out: Swap being unlocked from the pool
    swap_out = -1.0 * ts_data['tokenAmountOut']
    
    # Fee to be paid
    swap_fee = swap_in * params['transaction_fee']
    
    # Logic for mapping the input and output into tokens
    if ts_data['tokenInSym'] == 'UNI':
        fee_UNI = swap_fee
        fee_BAL = 0.0
        swap_UNI = swap_in
        swap_BAL = swap_out
    elif ts_data['tokenInSym'] == 'BAL':
        fee_UNI = 0.0
        fee_BAL = swap_fee
        swap_UNI = swap_out
        swap_BAL = swap_in
    else:
        raise Exception

    return {'fee_UNI': fee_UNI,
            'fee_BAL': fee_BAL,
            'swap_UNI': swap_UNI,
            'swap_BAL': swap_BAL,
            'timestamp': timestamp}

## 4. State Update Functions

In [10]:
def s_timestamp(params,
                      substep,
                      state_history,
                      previous_state,
                      policy_input):
    value = policy_input['timestamp']
    return ('timestamp', value)

def s_cumulative_fee_UNI(params,
                              substep,
                              state_history,
                              previous_state,
                              policy_input):
    variable = 'cumulative_fee_UNI'
    fee = policy_input['fee_UNI']
    value = previous_state['cumulative_fee_UNI'] + fee 
    return (variable, value)


def s_cumulative_fee_BAL(params,
                              substep,
                              state_history,
                              previous_state,
                              policy_input):
    variable = 'cumulative_fee_BAL'
    fee = policy_input['fee_BAL']
    value = previous_state['cumulative_fee_BAL'] + fee 
    return (variable, value)


def s_cumulative_swap_UNI(params,
                              substep,
                              state_history,
                              previous_state,
                              policy_input):
    variable = 'cumulative_swap_UNI'
    swap = policy_input['swap_UNI']
    value = previous_state['cumulative_swap_UNI'] + swap
    return (variable, value)


def s_cumulative_swap_BAL(params,
                              substep,
                              state_history,
                              previous_state,
                              policy_input):
    variable = 'cumulative_swap_BAL'
    swap = policy_input['swap_BAL']
    value = previous_state['cumulative_swap_BAL'] + swap
    return (variable, value)

## 5. Partial State Update Blocks

In [11]:
partial_state_update_blocks = [
    {
        'policies': {
            'policy_swap': p_swap
        },
        'variables': {
            'timestamp': s_timestamp,
            'cumulative_fee_UNI': s_cumulative_fee_UNI,
            'cumulative_fee_BAL': s_cumulative_fee_BAL,
            'cumulative_swap_UNI': s_cumulative_swap_UNI,
            'cumulative_swap_BAL': s_cumulative_swap_BAL
        }
    }
]

# Simulation

## 6. Configuration

In [12]:
sim_config = config_sim({
    "N": 1, # the number of times we'll run the simulation ("Monte Carlo runs")
    "T": range(len(df)), # the number of timesteps the simulation will run for
    "M": system_params # the parameters of the system
})

In [13]:
del configs[:] # Clear any prior configs

In [14]:
experiment = Experiment()
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

## 7. Execution

In [15]:
exec_context = ExecutionContext()
simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (100, 2, 2, 4)
Execution Method: local_simulations
SimIDs   : [0, 0]
SubsetIDs: [0, 1]
Ns       : [0, 1]
ExpIDs   : [0, 0]
Execution Mode: parallelized
Total execution time: 0.04s


## 8. Output Preparation

In [16]:
simulation_result = pd.DataFrame(raw_result)
simulation_result.head(5)

,cumulative_fee_BAL,cumulative_fee_UNI,cumulative_swap_BAL,cumulative_swap_UNI,simulation,subset,run,substep,timestep,timestamp
0,0.000000,0.000000,0.000000,0.000000,0,0,1,0,0,NaT
1,0.000000,2.131631,-78.928218,213.163101,0,0,1,1,1,2020-09-19 15:11:26
2,0.536815,2.131631,-25.246706,27.289805,0,0,1,1,2,2020-09-21 18:58:57
3,1.057537,2.131631,26.825501,-129.389807,0,0,1,1,3,2020-09-24 00:26:18
4,1.057537,4.169322,-43.109731,74.379302,0,0,1,1,4,2020-09-24 12:54:53


## 9. Analysis

In [17]:
# Visualize how much transaction fees were paid over time on each token
px.line(simulation_result,
           x='timestamp',
           y=['cumulative_fee_BAL', 'cumulative_fee_UNI'],
           facet_row='subset')

In [18]:
# Visualize the cumulative swaps:
# * Positive means that the pool has more tokens than initially
# * Negative means the opposite
px.line(simulation_result,
           x='timestamp',
           y=['cumulative_swap_BAL', 'cumulative_swap_UNI'],
           facet_row='subset')